In [1]:
import os
import sys
import numpy as np
os.environ["THEANO_FLAGS"] = "device=cpu"
sys.path.append(os.getcwd()+"/..")
from lightjob.cli import load_db
from collections import Counter
import glob
db = load_db()

In [2]:
def hash_matrix_to_int(hm):
    cnt = Counter(hm)
    s = np.argsort(cnt.values())[::-1]
    K = cnt.keys()
    K = [K[s[i]] for i in range(len(K))]
    K_to_int = {k: i + 1 for i, k in enumerate(K)}
    x = [K_to_int[v] for v in hm]
    x = np.array(x)
    return x


def unique_indices(hm):
    K = {}
    for i, h in enumerate(hm):
        if h not in K:
            K[h] = i
    return K.values()

def get_images(filenames):
    from skimage.io import imread
    for f in filenames:
        im = imread(f)
        im = 2 * (im / im.max()) - 1
        assert set(im.flatten().tolist()) <= set([1, -1]), set(im.flatten().tolist())
        yield im

In [3]:
s = 'd6ada23433955628d3a893310d3287f5'
job = db.get_job_by_summary(s)
job_folder = "../jobs/results/{}".format(s)
hash_matrix_filename = os.path.join(job_folder, "csv", "hashmatrix.npy")
hash_matrix = np.load(hash_matrix_filename)
x = hash_matrix_to_int(hash_matrix)

In [4]:
filenames = glob.glob(os.path.join(job_folder, 'final', '*.png'))
filenames = sorted(filenames)
indices = unique_indices(hash_matrix)
filenames = [filenames[ind] for ind in indices]

In [5]:
X = []
for im in get_images(filenames):
    X.append([im])
X = np.concatenate(X, axis=0)
X = X.reshape((X.shape[0], -1))

In [6]:
from sklearn.manifold import Isomap

In [7]:
manif = Isomap(n_components=2)
manif.fit(X)

Isomap(eigen_solver='auto', max_iter=None, n_components=2, n_neighbors=5,
    neighbors_algorithm='auto', path_method='auto', tol=0)